# NOTE TO SELF 4/10:: YOU NEED (1) NORMALIZE data, then TO find out what size convolution layers to use and find out 

In [1]:
from utils import * 
import utils as utils

training_df = pd.read_csv("data/training.csv").iloc[:500]
lookup_df = pd.read_csv("data/IdLookupTable.csv")
training_df.fillna(method = 'ffill',inplace = True)

In [2]:
train_X, train_Y, val_X, val_Y = create_train_test_sets_nchw(training_df)

In [3]:
train_X.shape, train_Y.shape, val_X.shape, val_Y.shape

(torch.Size([400, 1, 96, 96]),
 torch.Size([400, 30]),
 torch.Size([100, 1, 96, 96]),
 torch.Size([100, 30]))

In [4]:
type(train_X[0][0][0][0])

torch.Tensor

In [5]:
type(train_Y[0][0])

torch.Tensor

## Using torch dataloader and dataset class to format data

In [6]:
from torch.utils.data import Dataset, DataLoader

In [7]:
# ?DataLoader
# ??Dataset

In [11]:
class KeypointsDataset(Dataset):
    def __init__(self, images, keypoints):
        self.images = images.float()
        self.keypoints = keypoints.float()

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image = self.images[index]
        keypoint = self.keypoints[index]
        return image, keypoint

In [12]:
val_X.shape, val_Y.shape

(torch.Size([100, 1, 96, 96]), torch.Size([100, 30]))

In [13]:
dataset = KeypointsDataset(train_X, train_Y)
data_loader = DataLoader(dataset, batch_size=64,shuffle=False)

In [14]:
dataset

In [15]:
data_loader

In [16]:
for idx,batch in enumerate(data_loader):
    print("batch size: ", batch[0].size())
    print("batch idx: ",  idx)
    print("batch label: " , batch[1])
    break

batch size:  torch.Size([64, 1, 96, 96])
batch idx:  0
batch label:  tensor([[66.0336, 39.0023, 30.2270,  ..., 72.9355, 43.1307, 84.4858],
        [64.3329, 34.9701, 29.9493,  ..., 70.2666, 45.4679, 85.4802],
        [65.0571, 34.9096, 30.9038,  ..., 70.1918, 47.2749, 78.6594],
        ...,
        [65.4344, 36.8582, 29.2612,  ..., 64.6193, 46.6466, 83.6869],
        [65.4425, 36.5345, 29.1397,  ..., 68.2994, 47.2911, 80.6751],
        [65.7883, 34.7457, 26.6281,  ..., 70.4505, 47.6476, 81.9679]])


In [19]:
for idx, (data,target) in enumerate(data_loader):
    print(data.shape)
    print(target.shape)
    break

torch.Size([64, 1, 96, 96])
torch.Size([64, 30])


In [20]:
resnet = models.resnet18(pretrained=True)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [21]:
# print(model) #Show model architecture
resnet.parameters()

<generator object Module.parameters at 0x7f1c97130120>

In [22]:
# Seeing the parameters 

# for name, param in model.conv1.named_parameters():
#     if param.requires_grad:
#         print(name, param.data)

In [28]:
# Check the input and output shapes

for X_batch, y_batch in data_loader:
    print(X_batch.shape, y_batch.shape)
    break

torch.Size([64, 1, 96, 96]) torch.Size([64, 30])


## Very Simple CNN Model

In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

In [36]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 24 * 24, 30)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 24 * 24)
        x = self.fc1(x)
        return x

 

In [37]:
type(resnet)


torchvision.models.resnet.ResNet

In [38]:
simple_cnn = SimpleCNN()

In [39]:
type(simple_cnn)


__main__.SimpleCNN

In [40]:
# Train the model
n_epochs = 200
loss_fn = nn.BCELoss()
optimizer = optim.SGD(simple_cnn.parameters(), lr=0.1)
simple_cnn.train()
for epoch in range(n_epochs):
    for X_batch, y_batch in data_loader:
        y_pred = simple_cnn(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
 

RuntimeError: all elements of input should be between 0 and 1

In [ ]:
# evaluate accuracy after training
model.eval()
y_pred = model(X_test)
acc = (y_pred.round() == y_test).float().mean()
acc = float(acc)
print("Model accuracy: %.2f%%" % (acc*100))